# 03 - Tier 2 Benchmark (A100 GPU)

Run Tier 2 models (7B parameters) on the stratified sample set.
Requires Colab Pro with A100 GPU.

## Setup
1. Upload `colab_data.zip` to your Google Drive root
2. Select **A100 GPU** runtime in Colab
3. Run all cells in order

**Runtime**: ~1 hour for all Tier 2 models on 100 pages

In [ ]:
# === Colab Setup: clone repo, install deps, unpack data ===
!git clone https://github.com/srepho/OCR_Opensource.git 2>/dev/null || echo "Already cloned"
%cd OCR_Opensource
!pip install -q -e .
!pip install -q -r requirements/colab_tier2.txt

# Mount Google Drive and unpack data
from google.colab import drive
drive.mount('/content/drive')

import zipfile, os
DATA_ZIP = '/content/drive/MyDrive/colab_data.zip'
if os.path.exists(DATA_ZIP):
    with zipfile.ZipFile(DATA_ZIP, 'r') as zf:
        zf.extractall('.')
    print(f"Unpacked data from {DATA_ZIP}")
else:
    print(f"WARNING: {DATA_ZIP} not found - upload colab_data.zip to Drive root")

In [ ]:
import sys, torch
from pathlib import Path

PROJECT_ROOT = Path(".").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
if torch.cuda.is_available():
    vram_gb = torch.cuda.get_device_properties(0).total_mem / 1e9
    print(f"VRAM: {vram_gb:.1f} GB")
    assert vram_gb >= 16, f"Need >= 16GB VRAM for Tier 2, have {vram_gb:.1f}GB"

assert Path("data/sample_sets/stratified_100.json").exists(), "Sample sets missing"
print("Data check OK")

In [ ]:
import yaml

with open("config/benchmark_config.yaml") as f:
    config = yaml.safe_load(f)

SAMPLE_SET = "data/sample_sets/stratified_100.json"
IMAGE_DIR = config["paths"]["image_dir"]
OUTPUT_DIR = config["paths"]["raw_outputs_dir"]

TIER2_MODELS = [
    "olmocr",
    "rolmocr",
    "qwen25_vl",
    "chandra",
]

print(f"Will run {len(TIER2_MODELS)} models on {SAMPLE_SET}")

In [ ]:
from src.pipeline.runner import run_model_on_sample_set, load_model_registry

registry = load_model_registry("config/model_registry.yaml")
profiles = {}

for model_key in TIER2_MODELS:
    print(f"\n{'='*60}")
    print(f"Running: {model_key}")
    print(f"{'='*60}")
    try:
        profile = run_model_on_sample_set(
            model_key=model_key,
            sample_set_path=SAMPLE_SET,
            image_dir=IMAGE_DIR,
            output_dir=OUTPUT_DIR,
            registry=registry,
            device="cuda",
            skip_existing=True,
        )
        profiles[model_key] = profile
    except Exception as e:
        print(f"FAILED: {e}")
        import traceback
        traceback.print_exc()
    
    # Clear GPU memory between models
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        import gc; gc.collect()

In [ ]:
# Summary of all runs
import pandas as pd

rows = []
for model_key, profile in profiles.items():
    rows.append({
        "model": model_key,
        "pages": profile.total_pages,
        "avg_sec/page": round(profile.avg_time_per_page, 2),
        "pages/min": round(profile.pages_per_minute, 1),
        "peak_GPU_MB": round(profile.peak_gpu_memory_mb, 0),
    })

df = pd.DataFrame(rows)
print(df.to_string(index=False))

# Save results back to Drive
import shutil
DRIVE_RESULTS = '/content/drive/MyDrive/ocr_results'
shutil.copytree('results/raw_outputs', f'{DRIVE_RESULTS}/raw_outputs', dirs_exist_ok=True)
print(f"\nResults saved to {DRIVE_RESULTS}")